In [1]:
import pandas as pd
import numpy as np
import requests
import warnings
import os 
import pendulum

from tqdm import tqdm, trange
from datetime import datetime, timedelta
from functools import partial

from google.cloud import bigquery, storage

# from utils.gcp import (create_if_not_exist_google_storage, get_gcp_file_count,
#                        get_gcp_total_size, upload_to_bucket,
#                        validate_bucket_files)
# from utils.slack import on_failure, on_success
# from utils.util import make_directory

# from airflow import DAG
# from airflow.operators.dummy import DummyOperator
# from airflow.operators.python import PythonOperator

pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
pd.set_option('mode.chained_assignment',  None) 

warnings.simplefilter(action='ignore', category=FutureWarning)

# os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "****"

In [2]:
## get JSON from API url

def get_json(depart_date, depart, arrive):
    
    url1 = (
        "https://air.jejupass.com/air/agent/b2c/AIR/INT/AIRINTSCH0100100010.k1?"
        + "initform=OW"
        + "&initCnt=0"
        + "&initMax=2"
        + "&domintgubun=I"
        + "&depdomintgbn=D"
        + "&tasktype=B2C"
        + "&servicecacheyn=Y"
        + "&secrchType=FARE"
        + "&maxprice="
        + "&availcount=250"
        + "&preferaircd="
        + "&depctycd=" + depart
        + "&depctycd="
        + "&depctycd="
        + "&depctycd="
        + "&depctynm="
        + "&depctynm="
        + "&depctynm="
        + "&depctynm="
        + "&arrctycd=" + arrive
        + "&arrctycd="
        + "&arrctycd="
        + "&arrctycd="
        + "&arrctynm="
        + "&arrctynm="
        + "&arrctynm="
        + "&arrctynm="
        + "&depdt=" + depart_date
        + "&depdt="
        + "&depdt="
        + "&depdt="
        + "&opencase=N"
        + "&opencase=N"
        + "&opencase=N"
        + "&openday="
        + "&openday="
        + "&openday="
        + "&opendayNm="
        + "&opendayNm="
        + "&opendayNm="
        + "&nonstop="
        + "&adtcount=1"
        + "&chdcount=0"
        + "&infcount=0"
        + "&cabinclass=Y"
        + "&research=false"
        + "&anywhereDate="
        + "&KSESID=air%3Ab2c%3ASELK138XF%3ASELK138XF%3A%3A00"
    )
    url2 = "https://air.jejupass.com/air/agent/b2c/AIR/INT/AIRINTSCH010010001001.k1jsn?servicecacheyn=Y&requestedfaretype=&KSESID=air%3Ab2c%3ASELK138XF%3ASELK138XF%3A%3A00"

    req = requests.Session()
    session = req.get(url1)
    session = session.cookies['JSESSIONID']
    cookie = {
        'JSESSIONID': session
    }    

    res = requests.get(url2, cookies = cookie)
    json_data = res.json()

    return json_data

In [3]:
## JSON parsing and get details

def get_data(json_data):

    ## Price info
    price_df = pd.DataFrame()

    price_list = list(json_data['totalResult']['itineraries'].keys())

    for i in price_list:

        df1 = pd.json_normalize(json_data['totalResult']['itineraries'][i])
        df2 = pd.json_normalize(json_data['totalResult']['itineraries'][i]['fares'][0]['prices'])
        df3 = pd.json_normalize(json_data['totalResult']['itineraries'][i]['fares'])
        
        df1 = df1[['id', 'airline', 'seat', 'via', 'routingIds']]
        df2 = df2[['price', 'originPrice', 'fuel', 'tax', 'tasf', 'que', 'etc', 'total', 'discount']]
        df3 = df3[['booking', 'cabin', 'cabinNm']]

        price_merge = pd.concat([df1, df2, df3], axis =1 )

        price_df = price_df.append(price_merge)

    price_df = price_df.reset_index(drop = True)

    ## routing info
    routing_df = pd.DataFrame()

    routing_list = list(json_data['totalResult']['routings'].keys())

    for i in routing_list:

        df = pd.json_normalize(json_data['totalResult']['routings'][i])
        routing_df = routing_df.append(df)

    routing_df = routing_df.reset_index(drop = True)

    ## Time info
    time_df = pd.DataFrame()

    time_list = list(json_data['totalResult']['segments'])

    for i in time_list:
        df = pd.json_normalize(json_data['totalResult']['segments'][i])
        time_df = time_df.append(df)
        
    time_df = time_df.reset_index(drop = True)

    return price_df, routing_df, time_df

In [4]:
## data processing

def data_processing(price_df, routing_df, time_df):

    price_df = price_df.query("via == 0") # Direct only
    price_df = price_df.reset_index(drop = True)
    for i in range(len(price_df)):
        price_df.loc[i, ['routingIds', 'booking', 'cabin', 'cabinNm']] = price_df.loc[i, ['routingIds', 'booking', 'cabin', 'cabinNm']].apply(lambda x: ','.join(map(str, x)))
        
    routing_df['segmentIds'] = pd.DataFrame(routing_df['segmentIds'].tolist())[0]
    routing_df = routing_df.drop(['via', 'fstamp', 'fminute', 'depTime', 'jtime', 'ctime', 'atime', 'flightDay', 'codeshare'], axis = 1)

    time_df = time_df.drop(['airline', 'ctime', 'jtime', 'depTimestamp', 'arrTimestamp', 'fstamp', 'fminute', 'depTime', 'stopovers', 'srvlist'], axis = 1)

    merge1 = pd.merge(price_df, routing_df, left_on = 'routingIds', right_on = 'id', how = 'left')
    merge2 = pd.merge(merge1, time_df, left_on = 'segmentIds', right_on = 'id', how = 'left')

    final_df = merge2.sort_values('total')
    final_df = final_df.reset_index(drop = True)

    final_df['depTime'] = final_df['depDate'].str.slice(8, 12)
    final_df['depDate'] = final_df['depDate'].str.slice(0, 8)
    final_df['arrTime'] = final_df['arrDate'].str.slice(8, 12)
    final_df['arrDate'] = final_df['arrDate'].str.slice(0, 8)

    final_df = final_df[[
        'depDate'
        , 'depTime'
        , 'arrDate'
        , 'arrTime'
        , 'depCity'
        , 'depCityNm'
        , 'arrCity'
        , 'arrCityNm'
        , 'depAirport'
        , 'depAirportNm'
        , 'arrAirport'
        , 'arrAirportNm'
        # , 'discount' # ?
        , 'airline'
        , 'flightNo'
        , 'equipment'
        , 'equipmentNm'
        , 'booking'
        , 'cabin'
        # , 'an' # ?
        , 'seat'
        , 'via'
        , 'originPrice'
        , 'price'
        , 'fuel'
        , 'tax'
        , 'tasf'
        , 'que'
        , 'etc'
        , 'total'
        ]]

    search_date = datetime.today().strftime('%Y%m%d')
    final_df['searchDate'] = search_date
    
    final_df = final_df.reset_index(drop = True)
    final_df = final_df.astype({
        'depDate': 'string'
        , 'depTime': 'string'
        , 'arrDate': 'string'
        , 'arrTime': 'string'
        , 'depCity': 'string'
        , 'depCityNm': 'string'
        , 'arrCity': 'string'
        , 'arrCityNm': 'string'
        , 'depAirport': 'string'
        , 'depAirportNm': 'string'
        , 'arrAirport': 'string'
        , 'arrAirportNm': 'string'
        , 'airline': 'string'
        , 'flightNo': 'string'
        , 'equipment': 'string'
        , 'equipmentNm': 'string'
        , 'booking': 'string'
        , 'cabin': 'string'
        , 'seat': int
        , 'via': int
        , 'originPrice': int
        , 'price': int
        , 'fuel': int
        , 'tax': int
        , 'tasf': int
        , 'que': int
        , 'etc': int
        , 'total': int
        , 'searchDate': 'string'
    })

    return final_df

In [5]:
## Main fuction

def airline_ticket_japan():
    
    kor_airpot = ['SEL', 'CJU', 'PUS', 'CJJ', 'KWJ', 'TAE']
    jap_airport = ['FUK', 'TYO', 'SPK', 'NGO', 'OKA', 'KIX']

    air_route = []

    for depart in kor_airpot:
        for arrive in jap_airport:
            air_route.append([depart, arrive])
            air_route.append([arrive, depart])

    final_df = pd.DataFrame()

    for i in trange(1, 91):
        date = datetime.today() + timedelta(i)
        date = date.strftime('%Y-%m-%d')

        for j in air_route:
            depart = j[0]
            arrive = j[1]

            json_data = get_json(date, depart, arrive)

            if json_data['totalResult']['criteria'] == None:
                print(date, depart, arrive, "is None")
                continue

            # try
            
            price_df, routing_df, time_df = get_data(json_data)
            total_df = data_processing(price_df, routing_df, time_df)

            # file_name = f"airline_japan_dep={depart}_arr={arrive}_depDate={date}.parquet"
            # env = "prd" # stg: 테스트
            # root_directory = "crawling_data/airline_japan"
            # try:
            #     upload_to_bucket(total_df, file_name, env, root_directory)
            # except Exception as e:
            #     print(e)
            #     print("Failed to upload to bucket")

            final_df = final_df.append(total_df)

    return final_df

In [6]:
kor_airpot = ['SEL', 'CJU', 'PUS', 'CJJ', 'KWJ', 'TAE']
jap_airport = ['FUK', 'TYO', 'SPK', 'NGO', 'OKA', 'KIX']

air_route = []

for depart in kor_airpot:
    for arrive in jap_airport:
        air_route.append([depart, arrive])
        air_route.append([arrive, depart])

In [ ]:
# ## Without Multithreading

# final_df = pd.DataFrame()

# for i in trange(1, 4):
#     date = datetime.today() + timedelta(i)
#     date = date.strftime('%Y-%m-%d')

#     for j in air_route:
#         depart = j[0]
#         arrive = j[1]

#         json_data = get_json(date, depart, arrive)

#         if json_data['totalResult']['criteria'] == None:
#             # print(date, depart, arrive, "is None")
#             continue
        
#         price_df, routing_df, time_df = get_data(json_data)
#         total_df = data_processing(price_df, routing_df, time_df)

#         final_df = final_df.append(total_df)

In [12]:
## Multi-Threading

def main(start, end):
    final_df = pd.DataFrame()

    for i in trange(start, end):
        date = datetime.today() + timedelta(i)
        date = date.strftime('%Y-%m-%d')

        for j in air_route:
            depart = j[0]
            arrive = j[1]

            json_data = get_json(date, depart, arrive)

            if json_data['totalResult']['criteria'] == None:
                # print(date, depart, arrive, "is None")
                continue
            
            price_df, routing_df, time_df = get_data(json_data)
            total_df = data_processing(price_df, routing_df, time_df)

            final_df = final_df.append(total_df)
            # to_oracle(final_df)

    return final_df

In [14]:
start, end = 1, 2

final_df = pd.DataFrame()

for i in trange(start, end):
    date = datetime.today() + timedelta(i)
    date = date.strftime('%Y-%m-%d')

    for j in air_route:
        depart = j[0]
        arrive = j[1]

        json_data = get_json(date, depart, arrive)

        if json_data['totalResult']['criteria'] == None:
            # print(date, depart, arrive, "is None")
            continue
        
        price_df, routing_df, time_df = get_data(json_data)
        total_df = data_processing(price_df, routing_df, time_df)

        final_df = final_df.append(total_df)

final_df

100%|██████████| 1/1 [02:42<00:00, 162.79s/it]


,depDate,depTime,arrDate,arrTime,depCity,depCityNm,arrCity,arrCityNm,depAirport,depAirportNm,arrAirport,arrAirportNm,airline,flightNo,equipment,equipmentNm,booking,cabin,seat,via,originPrice,price,fuel,tax,tasf,que,etc,total,searchDate
0,20230118,1440,20230118,1555,SEL,서울,FUK,후쿠오카,ICN,인천,FUK,후쿠오카,TW,0295,737,BOEING 737 ALL SERIES PASSENGER,S,M,7,0,120000,120000,18400,28000,0,18400,0,166400,20230117
1,20230118,1720,20230118,1845,SEL,서울,FUK,후쿠오카,ICN,인천,FUK,후쿠오카,LJ,0225,772,BOEING 777-200/200ER,K,M,3,0,144000,144000,19900,28000,0,0,19900,191900,20230117
2,20230118,1815,20230118,1935,SEL,서울,FUK,후쿠오카,ICN,인천,FUK,후쿠오카,OZ,0136,321,AIRBUS INDUSTRIE A321,Q,M,3,0,170000,170000,26500,28000,0,0,26500,224500,20230117
3,20230118,1200,20230118,1320,SEL,서울,FUK,후쿠오카,ICN,인천,FUK,후쿠오카,LJ,0223,738,BOEING 737-800,E,M,9,0,192000,192000,19900,28000,0,0,19900,239900,20230117
4,20230118,0650,20230118,0810,SEL,서울,FUK,후쿠오카,ICN,인천,FUK,후쿠오카,LJ,0221,738,BOEING 737-800,D,M,3,0,200000,200000,19900,28000,0,0,19900,247900,20230117
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,20230118,1955,20230118,2100,FUK,후쿠오카,TAE,대구,FUK,후쿠오카,TAE,대구,TW,0234,737,BOEING 737 ALL SERIES PASSENGER,Y,M,9,0,174400,174400,57200,19200,0,57200,0,250800,20230117
0,20230118,1100,20230118,1320,TAE,대구,TYO,도쿄,TAE,대구,NRT,나리타,TW,0201,737,BOEING 737 ALL SERIES PASSENGER,S,M,6,0,160000,160000,31500,23000,0,31500,0,214500,20230117
0,20230118,1420,20230118,1650,TYO,도쿄,TAE,대구,NRT,나리타,TAE,대구,TW,0202,737,BOEING 737 ALL SERIES PASSENGER,Z,M,6,0,213100,213100,57200,35600,0,57200,0,305900,20230117
0,20230118,1015,20230118,1145,TAE,대구,OSA,오사카,TAE,대구,KIX,오사카/간사이,TW,0287,737,BOEING 737 ALL SERIES PASSENGER,W,M,3,0,240000,240000,18400,23000,0,18400,0,281400,20230117


In [ ]:
from concurrent.futures import ThreadPoolExecutor

with ThreadPoolExecutor(max_workers = 3) as executor:

    x = 1

    t1 = executor.submit(test, x)
    # t2 = executor.submit(test, x+7)
    # t3 = executor.submit(test, x+14)

t1